In [1]:
!pip install kaggle

In [2]:
!touch kaggle.json
!echo '{"username":"pncnmnp","key":"50f30c114712a78ad4159645314bcde6"}' > kaggle.json
!cat kaggle.json

{"username":"pncnmnp","key":"50f30c114712a78ad4159645314bcde6"}


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets list -s boolq

ref                    title                 size  lastUpdated          downloadCount  
---------------------  --------------------  ----  -------------------  -------------  
averkij/boolq-dataset  Google BoolQ Dataset   4MB  2020-01-17 15:10:39            251  
parthplc/boolq-csv     Boolq Csv              3MB  2020-08-04 16:03:08             22  
xhlulu/superglue       SuperGLUE             56MB  2020-01-24 02:42:33             33  


In [5]:
!kaggle datasets download -d averkij/boolq-dataset

boolq-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip boolq-dataset.zip

Archive:  boolq-dataset.zip
replace dev.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dev.jsonl               
replace test.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: test.jsonl              
replace train.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: train.jsonl             


In [7]:
!ls -lah

total 15M
drwxr-xr-x 1 root root 4.0K Oct 17 20:30 .
drwxr-xr-x 1 root root 4.0K Oct 17 18:01 ..
-rw-r--r-- 1 root root 4.0M Oct 17 18:03 boolq-dataset.zip
drwxr-xr-x 4 root root 4.0K Oct  8 13:44 .config
-rw-r--r-- 1 root root 2.2M Jan 17  2020 dev.jsonl
-rw-r--r-- 1 root root   64 Oct 17 20:16 kaggle.json
drwxr-xr-x 1 root root 4.0K Oct  8 13:45 sample_data
-rw-r--r-- 1 root root 2.1M Jan 17  2020 test.jsonl
-rw-r--r-- 1 root root 6.3M Jan 17  2020 train.jsonl


In [8]:
import pandas as pd
import json

with open("train.jsonl", "r") as file:
  train_set = list(file)
  
train_set = pd.DataFrame([json.loads(x) for x in train_set])
train_set.head()

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...


In [9]:
with open("dev.jsonl", "r") as file:
  dev_set = list(file)
  
dev_set = pd.DataFrame([json.loads(x) for x in dev_set])

In [10]:
# Checking the True and False distribution

train_set["answer"].value_counts()

True     5874
False    3553
Name: answer, dtype: int64

In [11]:
# observing an entire training set
import random
from pprint import pprint
pprint(train_set.sample(n=1).to_dict())

{'answer': {3976: True},
 'passage': {3976: 'Oregon is a shall-issue concealed-carry state. and is '
                   'notable for having very few restrictions on where a '
                   'concealed firearm may be carried. Oregon also has '
                   'statewide preemption for its concealed-carry laws--with '
                   'limited exceptions, counties and cities cannot place '
                   'limits on the ability of people to carry concealed weapons '
                   'beyond those provided by state law.'},
 'question': {3976: 'is it legal to carry a gun in your car in oregon'},
 'title': {3976: 'Gun laws in Oregon'}}


In [12]:
# Get the pre-trained word vectors
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [13]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

import numpy

def sent_from_word_vec(sent):
    word_vecs = numpy.array([glove_vectors.get_vector(word) 
                    for word in tknzr.tokenize(sent) 
                        if word in glove_vectors.vocab])
    sent_vec = word_vecs.sum(axis=0)/len(word_vecs)
    return sent_vec

In [14]:
train_set["passage"] = train_set["passage"].apply(lambda sent: sent_from_word_vec(sent))
train_set["question"] = train_set["question"].apply(lambda sent: sent_from_word_vec(sent))
train_set["answer"] = train_set["answer"].apply(lambda answer: int(answer))
# Averaging the question and answer vectors for each row
# Scikit does not accept 3D arrays
train_set["X"] = numpy.average(train_set[["passage", "question"]], axis=1)

In [15]:
 train_set

,question,title,answer,passage,X
0,"[0.39294538, 0.08274612, -0.21711724, 0.064575...",Persian language,1,"[0.23099138, -0.17384271, 0.017014574, -0.0686...","[0.3119684, -0.045548297, -0.10005133, -0.0020..."
1,"[-0.054775283, 0.4831755, -0.29438272, 0.07330...",Good Samaritan law,1,"[0.021168206, 0.21281487, -0.29497483, 0.05622...","[-0.016803538, 0.34799516, -0.29467878, 0.0647..."
2,"[0.19552127, -0.10706744, 0.21627998, -0.28216...",Windows Movie Maker,1,"[0.31184742, -0.16116633, -0.02541366, 0.02326...","[0.25368434, -0.13411689, 0.09543316, -0.12945..."
3,"[0.0005991161, -0.13639699, 0.19951749, 0.3821...",Powdered sugar,1,"[-0.17693591, -0.022432448, -0.06497649, 0.391...","[-0.0881684, -0.07941472, 0.0672705, 0.3866076..."
4,"[0.51841664, 0.011734881, 0.336105, -0.1527143...",The Elder Scrolls Online,0,"[0.082923114, -0.033176657, 0.011410061, -0.04...","[0.30066988, -0.010720888, 0.17375752, -0.1006..."
...,...,...,...,...,...
9422,"[0.05144, 0.36529875, -0.35352126, -0.24321625...",United States district court,1,"[0.09013317, 0.07126655, -0.19489315, -0.17184...","[0.07078658, 0.21828264, -0.2742072, -0.207529..."
9423,"[-0.089347005, 0.62398595, -0.322856, 0.254350...",Landlord harassment,1,"[0.027521614, 0.25012004, -0.15202706, -0.0354...","[-0.030912695, 0.437053, -0.23744154, 0.109444..."
9424,"[-0.0039708912, 0.176761, -8.1250444e-05, -0.5...",Golden State Warriors,1,"[0.14300688, 0.12835333, 0.12741962, -0.200886...","[0.06951799, 0.15255716, 0.06366918, -0.371998..."
9425,"[-0.18069042, 0.2675443, 0.37324128, -0.363978...",List of Downton Abbey episodes,0,"[0.17481926, -0.12980554, 0.23540363, -0.16010...","[-0.002935581, 0.06886938, 0.30432245, -0.2620..."


In [16]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=0).fit(train_set["X"].to_list(), train_set["answer"].to_list())

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [17]:
dev_set["passage"] = dev_set["passage"].apply(lambda sent: sent_from_word_vec(sent))
dev_set["question"] = dev_set["question"].apply(lambda sent: sent_from_word_vec(sent))
dev_set["answer"] = dev_set["answer"].apply(lambda answer: int(answer))
# Averaging the question and answer vectors for each row
# Scikit does not accept 3D arrays
dev_set["X"] = numpy.average(dev_set[["passage", "question"]], axis=1)

clf.score(dev_set["X"].to_list(), dev_set["answer"].to_list())

0.6195718654434251

In [18]:
from sklearn.metrics import classification_report

predicted_values = clf.predict(dev_set["X"].to_list())
print(classification_report(dev_set["answer"].to_list(), predicted_values))

              precision    recall  f1-score   support

           0       0.49      0.20      0.28      1237
           1       0.64      0.88      0.74      2033

    accuracy                           0.62      3270
   macro avg       0.57      0.54      0.51      3270
weighted avg       0.59      0.62      0.57      3270

